In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, SimpleRNN, GRU
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/Users/dipanjanpaul/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import wandb
##run = wandb.init()
##config = run.config
config_hidden_nodes = 128
config_batch_size = 256
config_file = 'male.txt'
config_maxlen = 200
config_step = 3

In [3]:
text = io.open(config_file, encoding='utf-8').read()
##text = text.replace('\n', ' ')
chars = sorted(list(set(text)))

In [4]:
len(chars)

54

In [5]:
len(text)/3

6764.333333333333

In [6]:
chars[0:10]

['\n', '-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [7]:
text[0:10]

'Aamir\nAaro'

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
len(text)

20293

In [10]:
sentences = []
next_chars = []
for i in range(0, len(text) - config_maxlen, config_step):
    sentences.append(text[i: i + config_maxlen])
    next_chars.append(text[i + config_maxlen])

In [11]:
sentences[0:3]

['Aamir\nAaron\nAbbey\nAbbie\nAbbot\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdams\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nA',
 'ir\nAaron\nAbbey\nAbbie\nAbbot\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdams\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nAdol',
 'Aaron\nAbbey\nAbbie\nAbbot\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdams\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nAdolphu']

In [12]:
next_chars[0:3]

['d', 'p', 's']

In [13]:
x = np.zeros((len(sentences), config_maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [14]:
x.shape

(6698, 200, 54)

In [15]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



In [16]:
x[1]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [17]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [18]:
model = Sequential()
model.add(LSTM(128, input_shape=(config_maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="rmsprop")

In [19]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
class SampleText(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        start_index = random.randint(0, len(text) - config_maxlen - 1)

        for diversity in [0.5, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + config_maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(200):
                x_pred = np.zeros((1, config_maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [21]:
model.fit(x, y, batch_size=config_batch_size,
              epochs=100, callbacks=[SampleText()])

Epoch 1/100
6698/6698 [==============================] - 24s 4ms/step - loss: 3.4002

----- diversity: 0.5
----- Generating with seed: "s
Niles
Nils
Nilson
Niven
Noach
Noah
Noam
Noble
Noe
Noel
Nolan
Noland
Norbert
Norm
Norman
Normand
Normie
Norris
Northrop
Northrup
Norton
Norwood
Nunzio
Obadiah
Obadias
Oberon
Obie
Octavius
Odell
Odie
"
s
Niles
Nils
Nilson
Niven
Noach
Noah
Noam
Noble
Noe
Noel
Nolan
Noland
Norbert
Norm
Norman
Normand
Normie
Norris
Northrop
Northrup
Norton
Norwood
Nunzio
Obadiah
Obadias
Oberon
Obie
Octavius
Odell
Odie

eaet
e
r


oIal
a


io
nn
kh
ne
d


nhrnnoonn
n
nsic
ensso
DinreP
eaela
iecyrr
e
aRh
rr

nliaF

ao
sioe
lnnhnaao
l
ree
g
s
ie

enon
e
ar
o
srn
eee
elaoantnoe
irtnaa
rinm


ieRe
o
oi
ee
nai

ne

nedin

----- diversity: 1.2
----- Generating with seed: "s
Niles
Nils
Nilson
Niven
Noach
Noah
Noam
Noble
Noe
Noel
Nolan
Noland
Norbert
Norm
Norman
Normand
Normie
Norris
Northrop
Northrup
Norton
Norwood
Nunzio
Obadiah
Obadias
Oberon
Obie
Octavius
Odell
Odie
"
s
Niles


ioaf
ilnnnoaPOan
vewe
ooaa
y
DSLygei
phraFd
yonoSha
alay
oHSotTCaRoAEkvghMovPSe
iEdeoCr
osSbZigctrlin
Totdasl
aSwWny
tarn
Cu
RRlHus

hhegcIieilsae
bss
g
ghils
TnnV
Flm
Epoch 7/100
6698/6698 [==============================] - 23s 3ms/step - loss: 2.8924

----- diversity: 0.5
----- Generating with seed: "rt
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter
Pietro
Piggy
Pincas
Pinchas
Pincus
Piotr
Pip
Plato
Pooh
Porter
Poul
Powell
Praneetf
Prasad
Prasun
Prent
Prentice
Pren"
rt
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter
Pietro
Piggy
Pincas
Pinchas
Pincus
Piotr
Pip
Plato
Pooh
Porter
Poul
Powell
Praneetf
Prasad
Prasun
Prent
Prentice
Prene
ualieSee
oseooim
oonunl
glele
onii
Loat
kei
orncLiiiri
F
oril
lolo
oioooie
Willie
Fain
uiii
eelsuEil
leiaen
aMaoto
Rvale
oalilh
hirii
calicoaies
Fliii
Henooi
cirsinhanGmomoe

noloon
oii
cain
eolrn
M

----- diversity: 1.2
----- Generating with seed: "rt
Philip
Phillip
Phillipe
Phillipp
Phinea

RaymuJSld
SirnsSceorSeweio
RBgrgaton
u
Alie

Gc
ieen
CodettK
MZLir
FleDiiaenanVicalsoabBedcGridchoKoTldiiQi
FadieLIiain
Roglgobi
Kucdan
EgieucTgeFolobo
TonudiunGy
Suen
RevsimdcReeioU
pangii
sUicki
Ggrrm
LI
Epoch 13/100
6698/6698 [==============================] - 23s 3ms/step - loss: 2.4594

----- diversity: 0.5
----- Generating with seed: "ki
Rickie
Ricky
Rik
Rikki
Riley
Rinaldo
Ripley
Ritch
Ritchie
Roarke
Rob
Robb
Robbert
Robbie
Robert
Roberto
Robin
Robinson
Rochester
Rock
Rockwell
Rocky
Rod
Rodd
Roddie
Roddy
Roderic
Roderich
Roderick
"
ki
Rickie
Ricky
Rik
Rikki
Riley
Rinaldo
Ripley
Ritch
Ritchie
Roarke
Rob
Robb
Robbert
Robbie
Robert
Roberto
Robin
Robinson
Rochester
Rock
Rockwell
Rocky
Rod
Rodd
Roddie
Roddy
Roderic
Roderich
Roderick
Reudmomed
Rroid
Ruuddd
Roddd
RdfRdodeon
Roo
dood
Radv
Riod
Relluen
Roudn
Roney
Rusind
Randd
Radar
RRudddRmduy
Todod
Rem
duddd
Rdlft
Ruedtou
Rain
Renard
Radaro
Roon
RodRood
oodd
Raned
Ruduld
Smemudtad


----- diversity: 1.2
----- Generating with seed: "ki


Mathian
Mkrmolr
LoCfasm
Doliosisn
ooe
kort
Iiio

boalpo
Fodt
Nniz
ick
MabdbacerReu
NBcpaoro
Nod
Moorbood
RcoT
TeudRiudodo
TahneTee
Tuedd
Toborn
Tlkee
TRru
eto
Ropu
Toidd
Juusddon
TodTmeuTT
bigeO
sere
Tobue

Epoch 19/100
6698/6698 [==============================] - 23s 3ms/step - loss: 2.0998

----- diversity: 0.5
----- Generating with seed: "ki
Nikolai
Nikos
Niles
Nils
Nilson
Niven
Noach
Noah
Noam
Noble
Noe
Noel
Nolan
Noland
Norbert
Norm
Norman
Normand
Normie
Norris
Northrop
Northrup
Norton
Norwood
Nunzio
Obadiah
Obadias
Oberon
Obie
Octav"
ki
Nikolai
Nikos
Niles
Nils
Nilson
Niven
Noach
Noah
Noam
Noble
Noe
Noel
Nolan
Noland
Norbert
Norm
Norman
Normand
Normie
Norris
Northrop
Northrup
Norton
Norwood
Nunzio
Obadiah
Obadias
Oberon
Obie
Octav
Fimerer
Airie
Arillo
Frilli
Fnrill
Fleili
Ollic
Filk
Fllill
Flliicc
Ficclil
Filiillril
Fik
FiiFili
FFlliFli
FliliFc
Fliic
Fili
Frlcils
FicilililliF
Finc
Fililii
Filiis
Filill
Filiil
Ailce
Flillii
Fil

----- diversity: 1.2
----- Generating with seed: "ki

Barry
Manrry
Pakh
Blle
aok
Ciiy
Joslhy
Carnri
Jiriy
Uiien
Wiuidhimas
Fic
orool
NicoeO
BNiwen
Roon
WiTd
cim
eoic
Pibriilt
Niirdiyn
NskicRiin
Riluy
RiyldJiges
Wbrtso
Noi
io
link
Nei
WillpszaiRi
Fkieh
reitio

Epoch 25/100
6698/6698 [==============================] - 24s 4ms/step - loss: 1.8521

----- diversity: 0.5
----- Generating with seed: "on
Art
Arther
Arthur
Artie
Artur
Arturo
Arvie
Arvin
Arvind
Arvy
Ash
Ashby
Ashish
Ashley
Ashton
Aub
Aube
Aubert
Aubrey
Augie
August
Augustin
Augustine
Augusto
Augustus
Austen
Austin
Ave
Averell
Averil
"
on
Art
Arther
Arthur
Artie
Artur
Arturo
Arvie
Arvin
Arvind
Arvy
Ash
Ashby
Ashish
Ashley
Ashton
Aub
Aube
Aubert
Aubrey
Augie
August
Augustin
Augustine
Augusto
Augustus
Austen
Austin
Ave
Averell
Averil
Allil
Alile
Allall
Alil
Alll
Alllil
Aallll
Ralili
Allli
All
Alllal
Alllll
Allel
oll
Fllet
Elllls
Ellul
Elan
Err
En
Enus
Ertiar
Elerd
Erriacer
Erres
Elierar
Errim
Errali
Errie
Erlerri
Errad
Frrior
Oerr

----- diversity: 1.2
----- Generating with seed: "on


...

Hallam
ElmeHammtin
Hlfemat
Hhviade
Hhimail
Hud
Halitoo
Hunt
Hymy
Hy
Hhgly
Huin
Ewhustmn
Dtin
Dittt
Ditty
Dutty
Duntoiy
Duwinnci
Duitn
Pundds
Aunn
Poyn
Dugie
Duuit
Quusig
Duadet

uutzato
Quiugst
Quinit
Qiuns
Epoch 73/100
6698/6698 [==============================] - 24s 4ms/step - loss: 0.8022

----- diversity: 0.5
----- Generating with seed: "Wyatt
Wye
Wylie
Wyn
Wyndham
Wynn
Wynton
Xavier
Xenos
Xerxes
Xever
Ximenes
Ximenez
Xymenes
Yaakov
Yacov
Yale
Yanaton
Yance
Yancey
Yancy
Yank
Yankee
Yard
Yardley
Yehudi
Yigal
Yule
Yuri
Yves
Zach
Zachari"
Wyatt
Wye
Wylie
Wyn
Wyndham
Wynn
Wynton
Xavier
Xenos
Xerxes
Xever
Ximenes
Ximenez
Xymenes
Yaakov
Yacov
Yale
Yanaton
Yance
Yancey
Yancy
Yank
Yankee
Yard
Yardley
Yehudi
Yigal
Yule
Yuri
Yves
Zach
Zacharia
Zachari
Zacharia
Zachay
Zancoa
Zear
Zanchare
Zancharie
Zacharie
Zacharie
Zacharie
Zacherial
Zacheray
Zerar
Zerar
Zer
Zacher
Zacharie
Zacharie
Zacharie
Zacharie
Zacharie
Zacharie
Zacharie
Zacharie
Za

----- diversity: 1.2
----- Generating with seed: "Wy

Berkoie
Berkammereor
Semoinok
Bermarre
Berrnat
Berrine
Berrite
Berrost
Brrke
Brkek
Bryve
Breny
Bervin
Bersiah
Aemerl
Bemam
Bemmmyy
Bemymm
Memmmmmymem
Hemllin
Hammemm
Hammmamlie
Halloml
Havamtel
HarullHMar
Epoch 79/100
6698/6698 [==============================] - 24s 4ms/step - loss: 0.7099

----- diversity: 0.5
----- Generating with seed: "oodman
Woodrow
Woody
Worden
Worth
Worthington
Worthy
Wright
Wyatan
Wyatt
Wye
Wylie
Wyn
Wyndham
Wynn
Wynton
Xavier
Xenos
Xerxes
Xever
Ximenes
Ximenez
Xymenes
Yaakov
Yacov
Yale
Yanaton
Yance
Yancey
Yanc"
oodman
Woodrow
Woody
Worden
Worth
Worthington
Worthy
Wright
Wyatan
Wyatt
Wye
Wylie
Wyn
Wyndham
Wynn
Wynton
Xavier
Xenos
Xerxes
Xever
Ximenes
Ximenez
Xymenes
Yaakov
Yacov
Yale
Yanaton
Yance
Yancey
Yanch
Yechanche
Jender
Jendon
Jennen
Jeen
Jeffie
Jeen
Jefffffffffffffffffffffffeffffffffffffffffffffffffffffffffffffffffffffffffffefffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffff

----- diversity: 1.2
----- Generating with seed: "oodm

Thoetorx
Terholwol
Thumrt
Thougyt
Thbubett
Tcicuis
Liccoccol
Liccek
Pickaucc
cicco
Picco
ccicccoocicccocceccoicaicoon
Piefnon
Ninco
Dittonoit
Dicoy
Diay
Doccas
Doicce
Docic
Dicceciociccock
Oicoelcoccocc
Epoch 85/100
6698/6698 [==============================] - 24s 4ms/step - loss: 0.6151

----- diversity: 0.5
----- Generating with seed: "
Lamont
Lance
Lancelot
Lane
Langston
Lanny
Larry
Lars
Laurance
Lauren
Laurence
Laurens
Laurent
Laurie
Lawerence
Lawrence
Lawson
Lawton
Lay
Layton
Lazar
Lazare
Lazaro
Lazarus
Lazlo
Lee
Lefty
Leif
Leigh"

Lamont
Lance
Lancelot
Lane
Langston
Lanny
Larry
Lars
Laurance
Lauren
Laurence
Laurens
Laurent
Laurie
Lawerence
Lawrence
Lawson
Lawton
Lay
Layton
Lazar
Lazare
Lazaro
Lazarus
Lazlo
Lee
Lefty
Leif
Leighellel
Lefffon
Lefffffofffefffefffffofffffeffffeffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffeofffffffffffffffffffffffffeffffefffffffffrelfferef
Wef

----- diversity: 1.2
----- Generating with seed: "
Lamon

Albrbincu
Argucke
Acgerku
Aubos
Austtie
Guris
Aswuste
Austan
Aunest
Astant
Awnastin
Avistini
Aptisoucs
Ausastur
Austrsteu
Auntin
Annntgs
Andiston
Ausiustit
uuviriudQ
uutid
Auchus
Atbuch
Audl
Aun
Audd
A
Epoch 91/100
6698/6698 [==============================] - 24s 4ms/step - loss: 0.5405

----- diversity: 0.5
----- Generating with seed: "ainwright
Wait
Waite
Waiter
Wake
Wakefield
Wald
Waldemar
Walden
Waldo
Waldon
Waleed
Walker
Wallace
Wallache
Wallas
Wallie
Wallis
Wally
Walsh
Walt
Walter
Walther
Walton
Wang
Ward
Warde
Warden
Ware
Wari"
ainwright
Wait
Waite
Waiter
Wake
Wakefield
Wald
Waldemar
Walden
Waldo
Waldon
Waleed
Walker
Wallace
Wallache
Wallas
Wallie
Wallis
Wally
Walsh
Walt
Walter
Walther
Walton
Wang
Ward
Warde
Warden
Ware
Warien
Warlen
Wally
Wallyn
Wasthangen
Wass
Waney
Wanson
Wasson
Wassans
Waston
Wanseas
Washaeshas
Walde
Washeashat
Washastan
Wastan
Wastan
Wastant
Wastean
Way
Wasdeas
Wande
Wald
Waldel
Walder
Walder
Walle


----- diversity: 1.2
----- Generating with seed: "ainwrig

Garot
Gor
Gorre
Grer
Greratt
reruas
Eearsi
Eas
Fammard
Farmis
Farlon
Farain
Frrawol
Garoron
Garron
Graivan
Gravevan
Gevit
Gev
Gevi
Gevi
Geev
Gewilme
Gereker
Gerido
Gervi
Germey
Gerin
Merhay
Hersa
Heriis
H
Epoch 97/100
6698/6698 [==============================] - 27s 4ms/step - loss: 0.4631

----- diversity: 0.5
----- Generating with seed: "n
Davoud
Davy
Dawson
Dean
Deane
Del
Delbert
Dell
Delmar
Demetre
Demetri
Demetris
Demetrius
Demosthenis
Denis
Dennie
Dennis
Denny
Derby
Derek
Derick
Derk
Derrek
Derrick
Derrin
Derrol
Derron
Deryl
Desmo"
n
Davoud
Davy
Dawson
Dean
Deane
Del
Delbert
Dell
Delmar
Demetre
Demetri
Demetris
Demetrius
Demosthenis
Denis
Dennie
Dennis
Denny
Derby
Derek
Derick
Derk
Derrek
Derrick
Derrin
Derrol
Derron
Deryl
Desmon
Dend
Dendie
Dennin
Denn
Denn
Denn
Dent
Dent
Dent
Dent
Dentin
Denntin
Dennin
Dennien
Dennil
Dernart
Dranke
Darrick
Darringar
Drarnichar
Darnie
Darrin
Darryn
Darrin
Darrin
Darrick
Darril
Darrin
Darric

----- diversity: 1.2
----- Generating with seed: "n
Da

In [82]:
start_index = random.randint(0, len(text) - config_maxlen - 1)
start_index

13564

In [84]:
generated = ''
sentence = text[start_index: start_index + config_maxlen]
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: "Pennie
Penny
Penrod
Pepe
Pepillo
Pepito
Perceval
Percival
Percy
Perry
Pete
Peter
Petey
Petr
Peyter
Peyton
Phil
Philbert
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter
"
Pennie
Penny
Penrod
Pepe
Pepillo
Pepito
Perceval
Percival
Percy
Perry
Pete
Peter
Petey
Petr
Peyter
Peyton
Phil
Philbert
Philip
Phillip
Phillipe
Phillipp
Phineas
Phip
Pierce
Pierre
Pierson
Piet
Pieter


In [85]:
len(sentence)

200

In [86]:
sentence

'Pennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\n'

In [88]:
x_pred = np.zeros((1, config_maxlen, len(chars)))
for t, char in enumerate(sentence):
    x_pred[0, t, char_indices[char]] = 1.
x_pred

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]])

In [99]:
preds = model.predict(x_pred, verbose=0)[0]
preds

array([2.9561197e-04, 2.2416114e-05, 8.7929402e-06, 8.1856502e-03,
       4.0501584e-03, 1.3970521e-03, 9.1291520e-05, 5.3150332e-05,
       5.9026564e-03, 6.6148076e-04, 1.3645522e-04, 2.3192408e-02,
       1.1398835e-02, 2.8561231e-02, 1.5119918e-01, 8.3863111e-03,
       3.5496548e-04, 6.7535806e-01, 2.5483403e-05, 1.6705006e-04,
       2.4389757e-03, 5.8176345e-04, 2.4308389e-04, 5.3082354e-04,
       8.5133081e-03, 8.5963365e-03, 6.4706436e-04, 3.6507923e-02,
       2.4629784e-05, 3.5612477e-06, 2.0376236e-04, 1.1800623e-06,
       1.1756299e-06, 1.6263000e-05, 4.0886385e-05, 8.5403258e-03,
       8.1454091e-05, 2.3979681e-05, 9.7814796e-04, 9.9374158e-03,
       1.8650548e-05, 7.1890520e-07, 7.0984861e-08, 1.3957510e-03,
       1.1709863e-03, 2.4838917e-07, 2.5554332e-06, 2.6005939e-06,
       1.7735313e-07, 5.2413310e-07, 2.2801462e-06, 2.1710994e-05,
       3.5998878e-06, 1.9968456e-05], dtype=float32)

In [100]:
diversity = 0.5
next_index = sample(preds, diversity)
next_char = indices_char[next_index]
next_char

'P'

In [102]:
generated += next_char
generated

'Pennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\nP'

In [103]:
sentence

'Pennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\n'

In [104]:
sentence = sentence[1:] + next_char
sentence

'ennie\nPenny\nPenrod\nPepe\nPepillo\nPepito\nPerceval\nPercival\nPercy\nPerry\nPete\nPeter\nPetey\nPetr\nPeyter\nPeyton\nPhil\nPhilbert\nPhilip\nPhillip\nPhillipe\nPhillipp\nPhineas\nPhip\nPierce\nPierre\nPierson\nPiet\nPieter\nP'

In [25]:
'Wade' in text

True

In [41]:
!python -m pip install win32com.client

  Could not find a version that satisfies the requirement win32com.client (from versions: )
No matching distribution found for win32com.client


In [33]:
!python -m pip install --upgrade pip3

/Users/dipanjanpaul/anaconda3/bin/python: No module named pip3


In [35]:
!python -m pip install --upgrade pip setuptools wheel

Requirement already up-to-date: pip in /Users/dipanjanpaul/anaconda3/lib/python3.6/site-packages (19.0.3)
    100% |████████████████████████████████| 583kB 10.4MB/s ta 0:00:01
nxviz 0.5.0 has requirement numpy==1.15.0, but you'll have numpy 1.16.2 which is incompatible.
nxviz 0.5.0 has requirement setuptools==40.0.0, but you'll have setuptools 40.8.0 which is incompatible.
  Found existing installation: setuptools 40.0.0
    Uninstalling setuptools-40.0.0:
      Successfully uninstalled setuptools-40.0.0
  Found existing installation: wheel 0.31.1
    Uninstalling wheel-0.31.1:
      Successfully uninstalled wheel-0.31.1


In [43]:
!python -m pip install /Users/dipanjanpaul/Downloads/pywin32-b224.tar

Processing /Users/dipanjanpaul/Downloads/pywin32-b224.tar
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "/private/var/folders/1r/wsdryzf53rxf6pxvnmd0gprh0000gn/T/pip-req-build-d5h2obsg/setup.py", line 76, in <module>
        import winreg # py3k
    ModuleNotFoundError: No module named 'winreg'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/1r/wsdryzf53rxf6pxvnmd0gprh0000gn/T/pip-req-build-d5h2obsg/setup.py", line 78, in <module>
        import _winreg as winreg # py2k
    ModuleNotFoundError: No module named '_winreg'
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/1r/wsdryzf53rxf6pxvnmd0gprh0000gn/T/pip-req-build-d5h2obsg/


In [42]:
import win32com.client

ModuleNotFoundError: No module named 'win32com'

In [46]:
!python -m pip install _winreg

Invalid requirement: '_winreg'

